## Adjusting accessibility statistics

In [3]:
import psycopg2
import configparser
import pandas as pd
import numpy as np

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

### Creating the function that calculates the adjustment required

In [6]:
SQL = """
DROP FUNCTION api_add_remove_catchments;

CREATE OR REPLACE FUNCTION api_add_remove_catchments(
    in_remove_hex_ids character[],
    in_add_hex_ids character[],		
	in_timeofday varchar,
	in_categorytype text,
    in_poi_category varchar
)
    RETURNS TABLE
            (
                h3id         character,                                
                adjustment        float
            )
    LANGUAGE plpgsql
AS
$addremovecatchments$
BEGIN
    RETURN QUERY
		--- Find all distinct catchment IDs where POIs are to be removed
        WITH 
            to_remove AS (
                select DISTINCT id FROM unnest(in_remove_hex_ids) m(id)
            ),
        remove_catchment_ids AS (
		        SELECT catchmentid,
				-COUNT(*) as sign
		        FROM catchments
                JOIN pois
                ON pois.h3id = catchments.originh3id 		        
		        JOIN to_remove ON catchments.originh3id = to_remove.id
                WHERE pois.category = in_poi_category
                GROUP BY catchmentid
		    ),
		--- Find all distinct catchment IDs where POIs are to be added
		add_catchment_ids AS (
			SELECT catchmentid,
				COUNT(*) as sign -- to allow for adding multiple POIs in the same hexagon ID
		        FROM catchments 		        
		        JOIN unnest(in_add_hex_ids) m(id) ON catchments.originh3id = m.id
                GROUP BY catchmentid
			),
			
		-- Merge them together
		all_catchment_ids AS (
			SELECT catchmentid, SUM(sign) as sign FROM 
			(SELECT * FROM add_catchment_ids UNION SELECT * FROM remove_catchment_ids) as t
			GROUP BY catchmentid
		)
            
		--- Obtain step 1 values to be adjusted		
		SELECT 
			step1.h3id, 			
			sum(ratio * sign) as ratio -- sign is + if adding, - if removing
			FROM step1_stats as step1
			JOIN all_catchment_ids ON step1.catchmentid = all_catchment_ids.catchmentid			
			WHERE 
				categorytype = in_categorytype
				AND
				timeofday = in_timeofday				
			GROUP BY step1.h3id;
		

END;
$addremovecatchments$;

"""

execute_sql(SQL)

### Check #1 - adding and removing the same catchment area

Case 1: removing and adding in a hex where only 1 POI of the type is present (Test case: Hospitals and clinics, H3ID: 8944c12a497ffff)

In [7]:
id = '8944c12a497ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(
	        array [%s], 
	        array [%s],	        
	        %s,
	        %s,
            %s)
        """, (id, id, 'morning', 'Race', 'Clinics and Hospitals'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values, 0))), "Not all values are zero - check!"
print("Test passed")
df.head()

Test passed


,hexid,adjustment
0,8944c12a483ffff,0.0
1,8944c138a7bffff,0.0
2,8944c138bcfffff,0.0
3,8944c138a27ffff,0.0
4,8944c138a6bffff,0.0


Case 1b: removing the same hex id 2 times should still result in zero adjustment

In [14]:
id = '8944c12a497ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(
	        %s, 
	        array [%s],	        
	        %s,
	        %s,
            %s)
        """, ([id]*2, id, 'morning', 'Race', 'Clinics and Hospitals'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values, 0))), "Not all values are zero - check!"
print("Test passed")
df.head()

Test passed


,hexid,adjustment
0,8944c12a483ffff,0.0
1,8944c138a7bffff,0.0
2,8944c138bcfffff,0.0
3,8944c138a27ffff,0.0
4,8944c138a6bffff,0.0


Case 2: removing and adding in a hex where only multiple POIs of the type is present (Test case: Restaurants, H3ID: 8944c12a497ffff, there are 14 of them)

In [8]:
id = '8944ccd9073ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(
	        array [%s], 
	        array [%s],	        
	        %s,
	        %s,
            %s)
        """, (id, id, 'morning', 'Race', 'Restaurants'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(df.adjustment.values < 0)), "Not all values are zero - check!"
print("Test passed")
df.head()

Test passed


,hexid,adjustment
0,8944ccd903bffff,-22.132556
1,8944ccd90a3ffff,-22.132556
2,8944ccd901bffff,-22.132556
3,8944ccd93dbffff,-22.132556
4,8944ccd9047ffff,-22.132556


Case 3: removing 1 POI where multiple exist and adding the same number (multiple) back should result in adjustment of zero.

In [9]:
id = '8944ccd9073ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("select COUNT(*) FROM pois where category = 'Restaurants' AND h3id = %s", (id, ))
        count = cur.fetchone()[0]
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(
	        %s,
	        %s,
	        %s,
	        %s,
            %s)
        """, ([id], [id]*count, 'morning', 'Race', 'Restaurants'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values,0))), "Not all values are zero - check!"
print("Test passed")
df.head()

Test passed


,hexid,adjustment
0,8944ccd93dbffff,0.0
1,8944ccd93d3ffff,0.0
2,8944ccd9297ffff,0.0
3,8944ccd92abffff,0.0
4,8944ccd906fffff,0.0


### Check #2 - adding a catchment area should result only in positive adjustments

In [10]:
id = ['8944c1a8133ffff']

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(	        
	        array [%s],
            %s, 	        
	        %s,
	        %s, %s)
        """, ("", [id], 'morning', 'Race', 'Restaurants'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values > 0, True))), "Not all values are larger than zero - check!"
assert (np.all(np.isclose(df.adjustment.values - df.adjustment.values[0], 0))), "Not all values are identical - check!"
print("Tests passed")
df.head()

Tests passed


,hexid,adjustment
0,8944c1a8e93ffff,0.07674
1,8944c1a83cfffff,0.07674
2,8944c1a164fffff,0.07674
3,8944c1aaa4fffff,0.07674
4,8944c1a9cd3ffff,0.07674


Case 2: adding 2 catchment areas that are overlapping should result in positive adjustments only, but 3 unique values.

In [11]:
ids = ['8944c1a8177ffff', '8944c1a812bffff']

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(	        
	        array [%s],
            %s, 	        
	        %s,
	        %s, %s)
        """, ("", ids, 'morning', 'Race', 'Restaurants'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values > 0, True))), "Not all values are larger than zero - check!"
assert (len(set(np.round(df.adjustment.values, 6))) == 3), "All values are identical - check!"
print("Tests passed")
set(np.round(df.adjustment.values, 6))

Tests passed


{0.081529, 0.108891, 0.19042}

### Check #3 - removing catchment area should result in identical negative adjustment everywhere

In [12]:
id = '8944c106d37ffff'

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        SELECT *
            FROM api_add_remove_catchments(	        	        
            array [%s], 
            array [%s],	        
	        %s,
	        %s,
            %s)
        """, (id, "", 'morning', 'Race', 'Restaurants'))
        res = cur.fetchall()

df = pd.DataFrame(res, columns=['hexid', 'adjustment'])
assert (np.all(np.isclose(df.adjustment.values < 0, True))), "Not all values are smaller than zero - check!"
assert (np.all(np.isclose(df.adjustment.values - df.adjustment.values[0], 0))), "Not all values are identical - check!"
print("Tests passed")

Tests passed
